<a href="https://colab.research.google.com/github/bhavya26082004/python/blob/main/AD4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install emot

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.5/61.5 kB 3.4 MB/s eta 0:00:00


In [2]:
text1= "What are you saying 😂, I am the boss 😎, and why are you so 😔"

In [3]:
import re
from emot.emo_unicode import UNICODE_EMOJI
from emot.emo_unicode import EMOTICONS_EMO

In [4]:
def converting_emojis(text):
  for emot in UNICODE_EMOJI:
    text=text.replace(emot, "_".join(UNICODE_EMOJI[emot].replace(",","").replace(":","").split()))
  return text
converting_emojis(text1)

'What are you saying face_with_tears_of_joy, I am the boss smiling_face_with_sunglasses, and why are you so pensive_face'

In [5]:
def emoji_removal(string):
    emoji_unicodes = re.compile("["
        u"\U0001F600-\U0001F64F"
        u"\U0001F300-\U0001F5FF"
        u"\U0001F680-\U0001F6FF"
        u"\U0001F1E0-\U0001F1FF"
        u"\U00002500-\U00002BEF"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642"
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"
        u"\u3030"
    "]+", flags=re.UNICODE)

    return emoji_unicodes.sub(r'', string)
emoji_removal(text1)

'What are you saying , I am the boss , and why are you so '

In [6]:
sent = ["What are you saying 😂", "I am the boss 😎", "why are you so 😔"]
import pandas as pd
df = pd.DataFrame({'text': sent})
df.head()


,text
0,What are you saying 😂
1,I am the boss 😎
2,why are you so 😔


In [7]:
import pandas as pd
dataset = pd.read_csv('/content/hate_speech.csv')
dataset.head()

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation


In [8]:
dataset.shape

(5242, 3)

In [9]:
dataset.label.value_counts()

,count
label,
0,3000
1,2242


In [10]:
for index, tweet in enumerate(dataset["tweet"][10:15]):
    print(index+1,".",tweet)

1 .  â #ireland consumer price index (mom) climbed from previous 0.2% to 0.5% in may   #blog #silver #gold #forex
2 . we are so selfish. #orlando #standwithorlando #pulseshooting #orlandoshooting #biggerproblems #selfish #heabreaking   #values #love #
3 . i get to see my daddy today!!   #80days #gettingfed
4 . ouch...junior is angryð#got7 #junior #yugyoem   #omg 
5 . i am thankful for having a paner. #thankful #positive     


In [11]:
import re
def clean_text(text):
    text = re.sub(r'[^a-zA\']', '', text)
    text = re.sub(r'[^\\x00-\x7F]+', ' ', text)
    text = text.lower()
    return text

In [12]:
dataset['clean_text'] = dataset.tweet.apply(lambda x: clean_text(x))

In [13]:
dataset.head(10)

,id,label,tweet,clean_text
0,1,0,@user when a father is dysfunctional and is s...,userwhenafatherisdysfunctionalandissoselfishhe...
1,2,0,@user @user thanks for #lyft credit i can't us...,useruserthanksforlyftcreditican tusecausetheyd...
2,3,0,bihday your majesty,bihdayyourmajesty
3,4,0,#model i love u take with u all the time in ...,modeliloveutakewithuallthetimeinur
4,5,0,factsguide: society now #motivation,factsguidesocietynowmotivation
5,6,0,[2/2] huge fan fare and big talking before the...,hugefanfareandbigtalkingbeforetheyleavechaosan...
6,7,0,@user camping tomorrow @user @user @user @use...,usercampingtomorrowuseruseruseruseruseruseruse...
7,8,0,the next school year is the year for exams.ð...,thenextschoolyearistheyearforexamscan tthinkab...
8,9,0,we won!!! love the land!!! #allin #cavs #champ...,wewonlovethelandallincavschampionsclevelandcle...
9,10,0,@user @user welcome here ! i'm it's so #gr...,useruserwelcomeherei mit ssogr


In [15]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [16]:
from nltk.corpus import stopwords
len(stopwords.words('english'))

179

In [17]:
stop = stopwords.words('english')

In [18]:
def gen_freq(text):
    word_list = []
    for tw_words in text.split():
        word_list.extend(tw_words)
    word_freq = pd.Series(word_list).value_counts()
    word_freq = word_freq.drop(stop, errors='ignore')
    return word_freq

In [20]:
def any_neg(words):
  for word in words:
    if word in ['n', 'no', 'non', 'not'] or re.search(r"\wn't", word):
      return 1
    else:
      return 0

In [21]:
def any_rare(words, rare_100):
  for word in words:
    if word in rare_100:
      return 1
    else:
      return 0

In [22]:
def is_question(words):
  for word in words:
    if word in ['when', 'what', 'how', 'why', 'who']:
      return 1
    else:
      return 0

In [23]:
word_freq = gen_freq(dataset.clean_text.str)
rare_100 = word_freq[-100:]
dataset['word_count'] = dataset.clean_text.str.split().apply(lambda x: len(x))
dataset['any_neg']=dataset.clean_text.str.split().apply(lambda x: any_neg(x))
dataset['is_question']=dataset.clean_text.str.split().apply(lambda x: is_question(x))
dataset['any_rare']=dataset.clean_text.str.split().apply(lambda x: any_rare(x,rare_100))
dataset.clean_text.apply(lambda x: len(x))

,clean_text
0,79
1,95
2,17
3,34
4,30
...,...
5237,45
5238,64
5239,82
5240,60
